In [ ]:
!pip install streamlit pyngrok
!ngrok config add-authtoken 2zzqGw4XNUa1NaYpfzxXif9w20h_6MJdp8SKo9us71Vd89DhJ




Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [61]:
import os
from pyngrok import ngrok
from threading import Thread

# Write the Streamlit app code to app.py
app_code = """
import streamlit as st
import pandas as pd

stock_prices = {
    "AAPL": 180,
    "TSLA": 250,
    "GOOGL": 2700,
    "MSFT": 300,
    "AMZN": 3500
}

st.title("📈 Stock Portfolio Tracker")

st.write("Enter your stocks and quantities (one per line) in the format: STOCK_SYMBOL, QUANTITY")
stock_input = st.text_area("Example:\\nAAPL, 10\\nTSLA, 5\\nGOOGL, 2")

if st.button("Calculate Total Investment"):
    portfolio = {}
    total_value = 0
    invalid = []

    lines = stock_input.strip().split('\\n')
    for line in lines:
        try:
            stock, qty = line.split(",")
            stock = stock.strip().upper()
            qty = int(qty.strip())
            if stock not in stock_prices:
                invalid.append(stock)
                continue
            portfolio[stock] = portfolio.get(stock, 0) + qty
        except:
            st.error(f"Invalid input format in line: '{line}'. Use 'STOCK_SYMBOL, QUANTITY'")
            st.stop()

    if invalid:
        st.warning(f"Price info not available for: {', '.join(set(invalid))}")

    if portfolio:
        st.write("### Portfolio Summary:")
        data = []
        for stock, qty in portfolio.items():
            price = stock_prices[stock]
            value = price * qty
            total_value += value
            st.write(f"{stock}: {qty} shares × ${price} = ${value}")
            data.append({
                "Stock": stock,
                "Quantity": qty,
                "Price per Share": price,
                "Total Value": value
            })

        st.write(f"### Total Investment Value: ${total_value}")

        df = pd.DataFrame(data)
        df.loc['Total'] = ["", "", "", total_value]

        csv = df.to_csv(index=False).encode('utf-8')
        st.download_button(
            label="Download Portfolio Summary CSV",
            data=csv,
            file_name="portfolio_summary.csv",
            mime="text/csv"
        )
    else:
        st.info("Please enter at least one valid stock and quantity.")
"""

with open("app.py", "w") as f:
    f.write(app_code)

# Kill any old ngrok tunnels (just in case)
!pkill ngrok > /dev/null 2>&1 || echo "No existing ngrok process."
ngrok.kill()


# Function to run Streamlit app
def run_streamlit():
    os.system("streamlit run app.py")

# Start ngrok tunnel on port 8501
public_url = ngrok.connect(8501).public_url
print(f"🚀 Streamlit app live at: {public_url}")

# Run Streamlit app in background thread
Thread(target=run_streamlit, daemon=True).start()

No existing ngrok process.
🚀 Streamlit app live at: https://9cc8cea63125.ngrok-free.app
